In [0]:
%pip install "mlflow>=1.25.0"
%pip install keras
%pip install tensorflow

In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import mlflow.pyfunc
from mlflow.models.signature import infer_signature
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
import pandas as pd
import numpy as np

In [0]:
# --- BƯỚC 8 (ĐÃ SỬA): PIPELINE DỰ BÁO TỰ ĐỘNG ---
# (Tương thích với Unity Catalog Aliases)

import mlflow
import pandas as pd
from pyspark.sql.functions import col, when
import datetime

# --- 8.1. Các tham số cấu hình ---

# !!! QUAN TRỌNG: Sửa lại <CATALOG_NAME> và <SCHEMA_NAME> của bạn
UC_CATALOG_NAME = "hcmut"
UC_SCHEMA_NAME = "gold"
MODEL_NAME = "WeatherForecast_LSTM_MultiCity"
MODEL_REGISTRY_NAME = f"{UC_CATALOG_NAME}.{UC_SCHEMA_NAME}.{MODEL_NAME}"

# --- (SỬA 1) ---
# MODEL_STAGE = "Production" # <-- BỎ DÒNG NÀY
MODEL_ALIAS = "prod" # <-- THÊM DÒNG NÀY (Phải khớp với alias bạn đặt ở Bước 1)
# ---------------

# Tên bảng Delta Lake (nguồn)
SOURCE_TABLE = "hcmut.gold.fact_vn_weather_hourly"
# Tên bảng Delta Lake (đích) để lưu kết quả
TARGET_TABLE = "hcmut.gold.lstm_weather_24h"

# Các thành phố cần dự báo
CITIES_TO_FORECAST = ["Ho Chi Minh City", "Da Nang City", "Ha Noi City"]

# Các thông số của mô hình (PHẢI GIỐNG HỆT KHI HUẤN LUYỆN)
N_INPUT_HOURS = 48 
N_OUTPUT_HOURS = 24

numeric_cols = [
    "nr_temperature_2m", "nr_dew_point_2m", "nr_relative_humidity_2m",
    "nr_pressure_msl", "nr_precipitation", "nr_cloud_cover",
    "nr_wind_speed_10m", "nr_wind_direction_10m", "nr_sunshine_duration"
]
location_cols = ["loc_hcm", "loc_dn", "loc_hn"]
FEATURE_COLUMNS = numeric_cols + location_cols
BASE_COLUMNS_TO_SELECT = ["dt_date_record", "ds_location"] + numeric_cols

# --- 8.2. Tải Mô hình "All-in-One" từ Registry ---

# Set registry về Unity Catalog
mlflow.set_registry_uri("databricks-uc")

# --- (SỬA 2) ---
print(f"Đang tải mô hình '{MODEL_REGISTRY_NAME}' với bí danh (alias) '@{MODEL_ALIAS}'...")
# Cú pháp mới: models:/<model_name>@<alias>
model_uri = f"models:/{MODEL_REGISTRY_NAME}@{MODEL_ALIAS}"
# ---------------

try:
    loaded_model = mlflow.pyfunc.load_model(model_uri)
    print("--- Tải mô hình thành công ---")
except Exception as e:
    # --- (SỬA 3) ---
    print(f"LỖI: Không thể tải mô hình. Bạn đã đặt alias '@{MODEL_ALIAS}' cho phiên bản mới nhất chưa?")
    print(f"Chi tiết lỗi: {e}")
    dbutils.notebook.exit(f"Model loading failed. Check alias '@{MODEL_ALIAS}'.") # <-- KÍCH HOẠT DÒNG NÀY
    # ---------------

# --- 8.3. Lấy Dữ liệu, Xử lý và Dự báo (Lặp qua từng thành phố) ---

all_forecasts = [] 

for city_name in CITIES_TO_FORECAST:
    print(f"\n--- Đang xử lý cho: {city_name} ---")
    
    try:
        # Lấy N_INPUT_HOURS (ví dụ: 48) bản ghi mới nhất
        print(f"Đang lấy {N_INPUT_HOURS} giờ dữ liệu mới nhất...")
        latest_data_df = (
            spark.read.table(SOURCE_TABLE)
            .select(*BASE_COLUMNS_TO_SELECT)
            .withColumn("loc_hcm", when(col("ds_location") == "Ho Chi Minh City", 1).otherwise(0))
            .withColumn("loc_dn", when(col("ds_location") == "Da Nang City", 1).otherwise(0))
            .withColumn("loc_hn", when(col("ds_location") == "Ha Noi City", 1).otherwise(0))
            .filter(col("ds_location") == city_name)
            .select(["dt_date_record"] + FEATURE_COLUMNS) 
            .orderBy(col("dt_date_record").desc())
            .limit(N_INPUT_HOURS) 
            .orderBy(col("dt_date_record").asc()) 
        ).toPandas() 

        if latest_data_df.shape[0] != N_INPUT_HOURS:
            print(f"LỖI: Không tìm thấy đủ {N_INPUT_HOURS} giờ dữ liệu cho {city_name}. Tìm thấy {latest_data_df.shape[0]} giờ. Bỏ qua...")
            continue 

        print(f"Dữ liệu đầu vào (từ {latest_data_df['dt_date_record'].min()} đến {latest_data_df['dt_date_record'].max()})")

        # --- 8.4. Thực hiện Dự báo ---
        print("Đang thực hiện dự báo...")
    
        forecast_values = loaded_model.predict(latest_data_df[FEATURE_COLUMNS])

        print("--- Dự báo thành công ---")

        # --- 8.5. Xử lý Kết quả (Cho thành phố này) ---
        last_known_time = latest_data_df['dt_date_record'].max()
        forecast_index = [
            last_known_time + pd.Timedelta(hours=i + 1) for i in range(N_OUTPUT_HOURS)
        ]

        results_df = pd.DataFrame({
            'dt_forecast_time': forecast_index,
            'nr_predicted_temperature': forecast_values,
            'ds_location': city_name,
            'dt_model_run_time': datetime.datetime.now(datetime.timezone.utc),
            'ds_model_version_uri': model_uri # Lưu lại URI của mô hình đã dự báo
        })
        
        all_forecasts.append(results_df)

    except Exception as e:
        print(f"LỖI trong quá trình xử lý cho {city_name}: {e}")
        # (Nếu có lỗi ở đây, chúng ta vẫn tiếp tục với thành phố tiếp theo)

# --- 8.6. Lưu Tất cả Kết quả vào Delta Lake ---

if not all_forecasts:
    print("\nKhông có kết quả dự báo nào được tạo ra. Kết thúc.")
else:
    print(f"\n--- TỔNG HỢP KẾT QUẢ DỰ BÁO ({len(all_forecasts)} thành phố) ---")
    final_results_df = pd.concat(all_forecasts)
    display(final_results_df)

    print(f"Đang lưu tất cả kết quả vào bảng {TARGET_TABLE}...")
    
    try:
        results_spark_df = spark.createDataFrame(final_results_df)
        (results_spark_df.write
            .format("delta")
            .mode("append")
            .option("mergeSchema", "true") 
            .saveAsTable(TARGET_TABLE)
        )
        print("--- LƯU KẾT QUẢ VÀO DELTA LAKE THÀNH CÔNG ---")
    except Exception as e:
        print(f"LỖI khi lưu vào Delta Lake: {e}")